<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>Color Transfer by Linear Programming</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.

**NOTE:** Run the following script whenever running this script on a Google Colab.

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

**REMARK:** If working on your personal computer, you might want to install [Gurobi with an academic license](https://www.gurobi.com/academia/academic-program-and-licenses/). Gurobi is a commercial solver that is extremely fast in solving large ILP instances, much more faster than the free GLPK solver.

# 5. Color Transfer by Linear Programming
In this lab session, you have to write a **Linear Programming** model to solve the optimal color transfer problem.

The problem is defined as follows. Given two images *A* and *B*, that are two 3-dimensional matrices of size $n \times m \times 3$, we want to optimally transfer the color palette of image *A* into image *B* by using a subset of $N$ random pixels of the two images.

Note that for image *A*, we have $m \times n$ pixels which have associated a vector of three components *[Red, Green, Blue]* representing the three level of intensity for each color; each component of the RGB vector has a value in the range $[0,...,255]$. In practice, we represent every pixel has an element of $\mathbb{R}_+^3$. Note that to visualize colors in python we have to normalize the three color channels to value in $[0..1]$, and hence we divide each color channel by 255.

If we take two pixels $x,y \in \mathbb{R}_+^3$, we can measure their (color) RGB-distance using the Euclidean distance.

Now we ready to state the problem: Take $N$ pixels (points) from image $A$, denoted by $N_A$, and $N$ points (pixel) from image $B$, denoted by $N_b$, and find the optimal mapping $\pi$ of the points in $N_A$ to the points in $N_B$, that is $\pi : N_A \rightarrow N_B$, such that the overall sum of the (color) RGB-distances induced by the mapping is minimal.

**NOTE:** Once you have the optimal mapping $\pi : N_A \rightarrow N_B$, for each pixel $i$ of image $A$, you look for the closest pixel $j \in N_A$, and you replace the color of pixel $j$ with the color of pixel $\pi(i)$.

![Borgo Ticino](borgo.png)

**EXERCISE 5.1:** Formulate the problem of finding the optimal mapping as a LP problem and solve it with Pyomo.

## 5.1 Loading and showing images as numpy matrices
To load a .jpeg image into a matrix we can use the [imread](https://matplotlib.org/3.5.1/api/_as_gen/matplotlib.pyplot.imread.html) function of the [matplotlib](https://matplotlib.org/) library, that takes as input a string representing a filename, and it returns a [numpy](https://numpy.org/) matrix.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def LoadImage(filename):
    # Read and normalize image with color channel intesiti into the range [0..1]
    A = plt.imread(filename).astype(np.float64) / 255.0
    return A

A = LoadImage('../data/notte.jpg')
B = LoadImage('../data/borgo.jpg')

To check the size of the image, you can type

In [ ]:
print('size of A:', A.shape)
print('size of B:', B.shape)

And to draw the two original images:

In [ ]:
# Defin a function for convinience
def ShowImage(A):
    fig, ax = plt.subplots()
    plt.imshow(A)
    ax.autoscale()
    ax.margins(0.1)
    ax.set_aspect('equal', 'box')
    plt.axis('off')
    plt.show()
    
# Use the drawing function
ShowImage(A)
ShowImage(B)

## 5.2 Sampling pixels as point in $\mathbb{R}^3$
Given the two input images as two matrices, we can sample $N$ pixels with the following snippet, where we first vectorize the matrices (by stacking the columns) and then sampling the indeces of the pixel vector.

In [ ]:
def PointSamples(A, samples=100):
    n,m,_ = A.shape
    C = A.reshape(n*m,3)
    s = np.random.randint(0, n*m, samples)
    return C[s]

We can then display the two images as cloud of point in $\mathbb{R}^3$ using a [scatter plot](https://matplotlib.org/3.5.0/api/_as_gen/matplotlib.pyplot.scatter.html).

In [ ]:
def DisplayCloud(C):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    plt.scatter(x=C[:,0], y=C[:,1], zs=C[:,2], s=10.0, c=C[:] )
    plt.show()
    #plt.savefig('cloud1.pdf', bbox_inches = 'tight', pad_inches = 0)

In [ ]:
H1 = PointSamples(A, 100)
H2 = PointSamples(B, 100)

In [ ]:
DisplayCloud(H1)

In [ ]:
DisplayCloud(H2)

## 5.3 Finding a color mapping
Given two samples of $N$ pixels, we can find a random mapping of the color of the first image into the second by taking a random permutation of the pixel indeces.

In [ ]:
from random import shuffle

def RandomMapping(H1, H2):
    N = len(H1)
    I = [i for i in range(N)]
    shuffle(I) # Make a random shuffle of the elements of the list
    return I

CMAP = RandomMapping(H1, H2)

To compute the closest vector using an efficient function for [computing distances](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html), we can use the [scipy spatial](https://docs.scipy.org/doc/scipy/reference/spatial.html) library:

In [ ]:
# Find the close vector
from scipy.spatial.distance import cdist
def ClosestRGB(A, B):
    return np.argmin(cdist(A, B), axis=1)

Then, to transfer the color using the mapping given by CMAP:

In [ ]:
def TransferColor(B, H1, H2, CMAP):
    n,m,_ = B.shape
    C = B.reshape(n*m,3)
    Y = ClosestRGB(C, H1)
    H4 = np.array([H2[CMAP[i]] for i in Y])
    H5 = H4.reshape(n,m,3)
    ShowImage(H5)

In [ ]:
TransferColor(B, H1, H2, CMAP)

## 5.4 Exercise: Finding the optimal mapping
You have to write an LP model that solves the optimal mapping problem, given two samples of $N$ pixels of two different images. 

In the end you, have to return a permutation $\pi$ of the indices of $N$ pixels, such that the following quantity is minimized: 

$$\sum_{i=1}^N || N_A(i) - N_B(\pi(i)) ||$$

where $N_A$, $N_B$ are the sampled pixels from image $A$ and $B$.

In [ ]:
!wget http://www-dimat.unipv.it/gualandi/opt4ds/notte.jpg

In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory
from pyomo.environ import RangeSet, NonNegativeReals, Binary, minimize

def D(a,b):
    return np.linalg.norm(a-b)**2
    
# Complete the following snippet
def OptimalMapping(H1, H2):
    n = len(H1)
    m = len(H2)

    # TODO: Complete....
    mod = ConcreteModel()

    mod.I = RangeSet(0, n-1)
    mod.J = RangeSet(0, m-1)

    mod.pi = Var(mod.I, mod.J, within=NonNegativeReals)

    mod.obj = Objective(expr=sum(D(H1[i], H2[j])*mod.pi[i,j] for i,j in mod.pi ) )

    mod.X = Constraint(mod.I,
        rule = lambda m, i: sum(m.pi[i,j] for j in m.J) == 1)

    mod.Y = Constraint(mod.J, 
        rule = lambda m, j: sum(m.pi[i,j] for i in m.I) == 1)

    # Solve the model
    opt = SolverFactory("gurobi") 
    #opt.options['Method'] = 1
    #opt.options['Crossover'] = 0
    sol = opt.solve(mod, tee=False)

    # Get a JSON representation of the solution
    sol_json = sol.json_repn()
    #print("runtime:", sol_json['Solver'][0]['Wall time'])
#    SolverFactory('gurobi').solve(mod, tee=True)

    ColMap = []
    for i in mod.I:
        for j in mod.J:
            if mod.pi[i,j]() > 0.5:
                ColMap.append(j)
    return ColMap
    #for i in mod.I:
    #    Ls = []
    #    for j in mod.J:
    #        if 0.01 < mod.pi[i,j]():
    #            Ls.append( (j, mod.pi[i,j]()) )

        # Random sampling for fractional solutions
    #    Ps = np.array([v[1] for v in Ls])
    #    idx = np.random.choice(len(Ls), p=Ps/np.sum(Ps))
    #    ColMap.append(Ls[idx][0])
    
    #return ColMap

You can test your solution by running:

In [ ]:
from time import perf_counter
H1 = PointSamples(A, 1000)
H2 = PointSamples(B, 1000)

t0 = perf_counter()

CMAP = OptimalMapping(H2, H1)
TransferColor(B, H2, H1, CMAP)

print('Overall runtime:', perf_counter()-t0)

## 5.5 Working by hand with color palettes
In Python, it is quite straightforward to define a palette of colors, since you only need to define a vector of RGB points.

Suppose you want to select a color: you can go to the [RapidTable](https://www.rapidtables.com/web/color/RGB_Color.html) website and find the RGB values for your favorite color. Mine is bly, and here I can display a blue point:

In [ ]:
# Define a color
color = np.array([[0,128,255]])/255
# Plot the color
plt.scatter([1], [1], c=color, s=200)
plt.show()

Now, suppose I want to create a gradient of colors from dark blue to light blue:

In [ ]:
# To take a first color: https://www.rapidtables.com/web/color/RGB_Color.html
color = np.array([[0.0,i/255,1.0] for i in range(128-100,128+100, 20)])
# Plot the palettes
Xs = [1+i for i in range(len(color))]
Ys = [1 for _ in color]
plt.scatter(Xs,Ys, c=color, s=200)
plt.show()

**EXERCISE:** Modify the Optimal Mapping function so that it can take in input an image ($N$ random sample point) and a color palette you define *manually* representing $K < N$ colors. In output, you can have more than a single pixel from the image assigned (or mapped) to the same palette color.

In [ ]:
# Complete the following snippet
# H2 is a color palette with len(H2) < len(H1)
def PaletteMapping(H1, H2):
    N = len(H1)
    M = len(H2)
    
    mod = ConcreteModel()
    
    mod.I = RangeSet(0, N-1)
    mod.J = RangeSet(0, M-1)
    
    mod.x = Var(mod.I, mod.J, within=NonNegativeReals)
    
    mod.obj = Objective(expr=sum(D(H1[i], H2[j]) * mod.x[i,j] for i,j in mod.x))
    
    mod.A = Constraint(mod.I,
                       rule = lambda m, i: sum(m.x[i,j] for j in m.J) == M)

    mod.B = Constraint(mod.J,
                       rule = lambda m, j: sum(m.x[i,j] for i in m.I) == N)
    

    SolverFactory('gurobi').solve(mod, tee=True)
    
    ColMap = []

    for i in mod.I:
        for j in mod.J:
            if mod.x[i,j]() > 0.5:
                ColMap.append(j)
                
    return ColMap

In [ ]:
def BluPalette():
    # Pick a color: https://www.rapidtables.com/web/color/RGB_Color.html
    return np.array([i/255, (255-i)/255, i/255] for i in range(128-100,128+100, 1)])

In [ ]:
H1 = PointSamples(A, 200)
H2 = BluPalette()
print(H2.shape)
CMAP = PaletteMapping(H1, H2)
TransferColor(B, H1, H2, CMAP)

## 5.6 Using a better color space
Measuring the distance between two colors in the RGB space does not reflect the human perception of colors. For this reason, different color spaces where studied and standardized, as for instance, for the [CIELAB](https://en.wikipedia.org/wiki/Color_difference#CIELAB_%CE%94E*) standard. For an explanation of the three coordinates in the CIELAB color space, we refer to [wikipedia](https://en.wikipedia.org/wiki/CIELAB_color_space#CIELAB_coordinates).

The conversion from a RGB image to an $L*a*b$ image can be realized with the following function:

In [ ]:
# Taken from:
# https://stackoverflow.com/questions/13405956/convert-an-image-rgb-lab-with-python

def rgb2lab(inputColor):
    num = 0
    RGB = [0, 0, 0]
    for value in inputColor:
        value = float(value) / 255
        if value > 0.04045:
            value = ((value + 0.055 ) / 1.055)**2.4
        else:
            value = value / 12.92

        RGB[num] = value * 100
        num = num + 1

    XYZ = [0, 0, 0,]

    X = RGB [0] * 0.4124 + RGB [1] * 0.3576 + RGB [2] * 0.1805
    Y = RGB [0] * 0.2126 + RGB [1] * 0.7152 + RGB [2] * 0.0722
    Z = RGB [0] * 0.0193 + RGB [1] * 0.1192 + RGB [2] * 0.9505
    XYZ[0] = round(X, 4)
    XYZ[1] = round(Y, 4)
    XYZ[2] = round(Z, 4)

    XYZ[0] = float( XYZ[0] ) / 95.047         # ref_X =  95.047   Observer= 2°, Illuminant= D65
    XYZ[1] = float( XYZ[1] ) / 100.0          # ref_Y = 100.000
    XYZ[2] = float( XYZ[2] ) / 108.883        # ref_Z = 108.883

    num = 0
    for value in XYZ :
        if value > 0.008856 :
            value = value ** ( 0.3333333333333333 )
        else:
            value = ( 7.787 * value ) + ( 16 / 116 )

        XYZ[num] = value
        num = num + 1

    Lab = [0, 0, 0]

    L = ( 116 * XYZ[ 1 ] ) - 16
    a = 500 * ( XYZ[ 0 ] - XYZ[ 1 ] )
    b = 200 * ( XYZ[ 1 ] - XYZ[ 2 ] )

    Lab[0] = round( L, 4 )
    Lab[1] = round( a, 4 )
    Lab[2] = round( b, 4 )

    return Lab

To notice the difference between the two color spaces, look at the following code:

In [ ]:
H3 = np.array([rgb2lab(c) for c in H2])

In [ ]:
def DisplayCloudLab(C, color):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    plt.scatter(x=C[:,0], y=C[:,1], zs=C[:,2], s=10.0, c=color[:] )
    plt.show()

In [ ]:
DisplayCloud(H2)

In [ ]:
DisplayCloudLab(H3, H2)

**EXERCISE (optional):** Modify the Optimal Mapping function to replace the Euclidean cost in the RGB space, with the Euclidean distance in the $L*a*b$ space (that is, the *old* CIE76 standard). Remember that the final image must be stilla RGB image, you only compute the distance between a pair of pixels by using their color in the $L*a*b$ space.

In [ ]:
def OptimalLabMapping(H1, H2):
    N = len(H1)
    I = [i for i in range(N)]
    
    # COMPLETE WITH A PYOMO MODEL AND ELABORATE THE RESULTS
    
    return I

In [ ]:
CMAP = OptimalLabMapping(H1, H2)
TransferColor(B, H1, H2, CMAP)